<a href="https://colab.research.google.com/github/ask-1710/ABUSIVE-LANGUAGE-DETECTION-IN-URDU/blob/main/HASOC_AbusiveLanguageDetectionInUrdu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
BASE_DIR = '/content/drive/MyDrive/Datasets/HASOC-Urdhu/Abusive'
TRAIN_DIR = BASE_DIR+'/train.csv'
TEST_DIR = BASE_DIR+'/test.csv'

In [3]:
!pip install urduhack[tf]

     |████████████████████████████████| 105 kB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 12.8 MB/s 
     |████████████████████████████████| 1.1 MB 39.9 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [4]:
import urduhack

# Downloading models
urduhack.download()

98304/89078 [=================================] - 0s 0us/step


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm  
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, cross_val_score
import pandas as pd

In [6]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import f1_score

In [7]:
test_data = pd.read_csv(TEST_DIR)
train_data = pd.read_csv(TRAIN_DIR)

In [8]:
X_train = train_data['tweet'].values
Y_train = train_data['target'].values

In [9]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data['tweet'].values, train_data['target'].values, shuffle=True, test_size=0.4)

In [10]:
max_feature_num = 50000
vectorizer = TfidfVectorizer(max_features=max_feature_num, ngram_range=(1,10),  lowercase=True, max_df=50, analyzer='char')
train_vecs = vectorizer.fit_transform(X_train)

LR

In [11]:
LR = LogisticRegression(random_state=0)

In [12]:
LR.fit(train_vecs, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
data_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(train_data['tweet'].values)
result_pred=LR.predict(data_vecs)
actual_result=train_data['target'].values

In [14]:
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("ROC_AUC_score",roc_auc_score(actual_result, result_pred))

 F1-score 0.8074187018412254
ROC_AUC_score 0.8077663281315655


In [15]:
LR = LogisticRegression(random_state=0)
res = cross_validate(LR, train_vecs, Y_train, cv=5,  return_train_score=True)
score = cross_val_score(LR, train_vecs, Y_train, cv=5)
print(score.mean())

0.8138888888888889


Nu-SVC

In [16]:
SVM = svm.NuSVC(kernel='linear', probability=True, random_state=42)  # sigmoid -> 80.91 Linear -> 80.54

In [17]:
SVM.fit(train_vecs, Y_train)
result_pred=SVM.predict(data_vecs)
actual_result=train_data['target'].values
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))

 F1-score 0.8023781595971722
private ROC_AUC_score 0.8028108854441944


In [18]:
SVM = svm.NuSVC(kernel='linear', probability=True, random_state=42)
res = cross_validate(SVM, train_vecs, Y_train, cv=5,  return_train_score=True)
score = cross_val_score(SVM, train_vecs, Y_train, cv=5)
print(score.mean())

0.8034722222222224


KNN

In [19]:
KNN = KNeighborsClassifier(n_neighbors=50, weights='uniform', algorithm='auto')

In [20]:
KNN.fit(train_vecs, Y_train)
result_pred=KNN.predict(data_vecs)
actual_result=train_data['target'].values
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))

 F1-score 0.7119016709118646
private ROC_AUC_score 0.717472397802242


In [21]:
KNN = KNeighborsClassifier(n_neighbors=50, weights='uniform', algorithm='auto')
res = cross_validate(KNN, train_vecs, Y_train, cv=5,  return_train_score=True)
score = cross_val_score(KNN, train_vecs, Y_train, cv=5)
print(score.mean())

0.795138888888889


Sentence Transformers

In [22]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 2.8 MB 27.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 636 kB 24.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=8a20fadba33e3035e3fe05ff21a5a48029e6e5b3084cff27cfa7cd789c18192b
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


distiluse-base-multilingual-cased-v2

In [23]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/610 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [24]:
from urduhack.models.lemmatizer import lemmatizer

In [25]:
def lemmatizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str


In [26]:
train_data['lemmatized'] = train_data['tweet'].apply(lambda x: lemmatizeStr(x))

In [27]:
X_train = model.encode(train_data['lemmatized'].values, batch_size=32,show_progress_bar=True)
Y_train = train_data['target'].values

Batches:   0%|          | 0/75 [00:00<?, ?it/s]

In [28]:
xtrain,xval,ytrain,yval = train_test_split(X_train, Y_train, test_size=0.4, shuffle=True)

In [29]:
MLPD = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)
MLPD.fit(xtrain, ytrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(256, 128), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [30]:
result_pred=MLPD.predict(X_train)
actual_result=Y_train
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))

 F1-score 0.8908350012975628
private ROC_AUC_score 0.8908385775830636


In [31]:
MLPD = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)

In [33]:
res = cross_validate(MLPD, X_train, Y_train, cv=5,  return_train_score=True)
score = cross_val_score(MLPD, X_train, Y_train, cv=5)
print(score.mean())

0.7754166666666666


Paraphrase-xlm-r-multilingual-v1


In [34]:
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
train_data['lemmatized'] = train_data['tweet'].apply(lambda x:lemmatizeStr(x))

In [36]:
X_train = model.encode(train_data['lemmatized'].values, batch_size=32,show_progress_bar=True)
Y_train = train_data['target'].values

Batches:   0%|          | 0/75 [00:00<?, ?it/s]

In [37]:
xtrain,xval,ytrain,yval = train_test_split(X_train, Y_train, test_size=0.4, shuffle=True)

In [38]:
MLPP = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)
MLPP.fit(xtrain, ytrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(256, 128), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [39]:
result_pred=MLPP.predict(X_train)
actual_result=Y_train
print(" F1-score",f1_score(actual_result,result_pred, average='weighted'))
print("private ROC_AUC_score",roc_auc_score(actual_result, result_pred))

 F1-score 0.8924828235511518
private ROC_AUC_score 0.8923970938255947


In [40]:
MLPP = MLPClassifier(hidden_layer_sizes=(256,128),max_iter=300)

In [41]:
res = cross_validate(MLPP, X_train, Y_train, cv=5,  return_train_score=True)
score = cross_val_score(MLPP, X_train, Y_train, cv=5)
print(score.mean())

0.77125


SUBMISSIONS

In [31]:
SUBS = '/content/drive/MyDrive/Datasets/HASOC-Urdhu/Abusive/submissions'

In [32]:
def testing(clf, SUBSPATH):
  df = pd.DataFrame()
  df['id'] = test_data['id']
  pred = clf.predict_proba(test_vecs)
  df['target'] = [1 for _ in test_data['id'].values]
  df['score'] = [p[1] for p in pred]

  df.to_csv(SUBSPATH)

In [48]:
X_test = test_data['tweet']
test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(X_test)

In [50]:
testing(LR, SUBS+'/lr3.csv')

In [ ]:
testing(SVM,SUBS+'/nusvc4.csv')

In [ ]:
testing(KNN,SUBS+'/knn2.csv')

In [ ]:
testing(MLPD, SUBS+'/distiluseMLP1.csv')

In [ ]:
testing(MLPP, SUBS+'/paraphrase-xlm5.csv')

TESTING

In [ ]:
# TEST_AND_DIR = '/content/drive/MyDrive/Datasets/HASOC-Urdhu/Abusive/Abusive task.zip'
# directory_to_extract_to = '/content/drive/MyDrive/Datasets/HASOC-Urdhu/Abusive/results'
# import zipfile
# with zipfile.ZipFile(TEST_AND_DIR, 'r') as zip_ref:
#     zip_ref.extractall(directory_to_extract_to)

In [ ]:
def calc_label(row) :
  if row['score'] < 0.5:
    row['target'] = 0
  else:
    row['target'] = 1
  return row

In [ ]:
test_ans = pd.read_csv('/content/drive/MyDrive/Datasets/HASOC-Urdhu/Abusive/results/test_with_ans_and_split.csv', index_col=0)

In [ ]:
test_ans

,target,split
id,,
3024,0,public
3235,1,private
2227,0,public
2659,1,private
1311,1,public
...,...,...
2928,1,private
1928,1,private
1931,0,private


In [ ]:
len(np.where(test_ans['target'] == 0)[0])

537

In [ ]:
len(np.where(test_ans['target'] == 1)[0])

563

In [ ]:
# separate public and private 
private = []
public = []


In [ ]:
for index, row in test_ans.iterrows():
  if row['split'] == 'public':
    public.append(index)
  else:
    private.append(index)  


In [ ]:
test_private = test_ans.loc[private]

In [ ]:
test_public = test_ans.loc[public]

In [ ]:
def accuracy(y_pred, y_test):
  return (roc_auc_score(y_test, y_pred))

In [ ]:
def testing(filename):
  res = pd.read_csv(SUBS+filename, index_col=0)
  res = res.apply(lambda row: calc_label(row),axis=1)


  private_res = res.loc[private]
  public_res = res.loc[public]

  return accuracy(private_res['target'].values, test_private['target'].values), f1_score(private_res['target'].values, test_private['target'].values), accuracy(public_res['target'].values, test_public['target'].values), f1_score(public_res['target'], test_public['target'].values)

In [ ]:
testing('distiluseMLP1.csv') # private(roc accuracy, f1score) , public(roc acc, f1score)

(0.7422772415035329,
 0.7220543806646526,
 0.7090131118006205,
 0.6666666666666666)

In [ ]:
testing('knn2.csv')

(0.726659789256503, 0.7230989956958394, 0.7234010609548593, 0.7027027027027026)

In [ ]:
testing('lr3.csv')

(0.7213502648621654, 0.723943661971831, 0.7343108797918126, 0.7225130890052357)

In [ ]:
testing('nusvc4.csv')

(0.6890960299331089,
 0.6875891583452212,
 0.7112651386247623,
 0.6933333333333334)

In [ ]:
testing('paraphrase-xlm5.csv')

(0.7576245097798409, 0.7719298245614035, 0.6995796216594936, 0.689119170984456)